In [ ]:
import mne 
import matplotlib.pyplot as plt
from scipy.io import *


file_path = r"E:\ChristianMusaeus\Data\Eyes_closed_marked\10002_p01_epoched_60EpochsMarked.set"
epochs = mne.io.read_epochs_eeglab(file_path)
mat = loadmat(file_path, struct_as_record= False, squeeze_me=True)

print(epochs)

epochs.plot()
plt.show()

In [ ]:
epochs.info

### Power Spectral Density

In [ ]:
psd =epochs.compute_psd()

psd_data=psd.get_data()
print(psd_data.shape)
psd.plot()
plt.show()


### Alpha Power

In [ ]:
freq = psd.freqs

fmin, fmax = 8, 13

freq_indices = (freq >= fmin) & (freq <= fmax)

# absolute alpha power
psd_alpha = psd_data[:, :, freq_indices].sum(axis=2)

# total power (all frequencies)
psd_total = psd_data.sum(axis=2)

# relative alpha power
psd_relative_alpha = psd_alpha / psd_total

print(f"Total Power Shape: {psd_total}")
print(f"Alpha Power Shape: {psd_alpha}")

In [ ]:
print(f"Alpha Power for first 5 epochs:", psd_alpha[:5])

In [ ]:
alpha_power_avg = psd_alpha.mean(axis=0) # shape: (n_channels,)
relative_alpha_power_avg = psd_relative_alpha.mean(axis=0) # shape: (n_channels,)

In [ ]:
regions = {
    'frontal':['Fz', 'Fp1', 'Fp2', 'F3', 'F4'],
    'central':['Cz', 'C3', 'C4', 'Pz'],
    'occipital':['O1', 'O2', 'Oz']
}

channels = psd.info['ch_names']
frontal_indices = [i for i, ch in enumerate(channels) if ch in regions['frontal']]
central_indices = [i for i, ch in enumerate(channels) if ch in regions['central']]
occipital_indices = [i for i, ch in enumerate(channels) if ch in regions['occipital']]

### Topographical plot of alpha activity on the scalp

In [ ]:
fig, ax = plt.subplots()

mne.viz.plot_topomap(psd_alpha.mean(axis=0), epochs.info, show=False, axes=ax, cmap='coolwarm')

ax.set_title("Alpha Band Power (8-13 Hz)")

plt.show()

The blue color represents low values, meaning there is little or no alpha activity in these regions.

White epresents middle values or no significant difference, meaning the alpha activity in these regions is not as high as in the red regions but still above baseline or neutral

Red indicates high values, meaning the alpha activity is high in these regions

In [ ]:
eeg_array = epochs.get_data()
print(eeg_array.shape)

### Rejected epochs

In [ ]:
import scipy.io

data_scipy = scipy.io.loadmat(file_path, struct_as_record=False, squeeze_me=True)

print(data_scipy.keys())

In [ ]:
import numpy as np
rej = data_scipy['reject']
rejected=rej.rejmanual
rejected_np= np.array(rejected)
print(rejected_np.shape) #svarer til antallet af epochs, der svarer til antallet af sekunder, målingerne har varet. 1807 epochs svarer til ca 30 minutter, hvilket passer. 


### Channels

In [ ]:
channels_info = epochs.info['chs']

for ch in channels_info:
    print(f"Channel Name: {ch['ch_name']}")
    #print(f"Channel Type: {ch['kind']}")
    #print(f"Channel Position: {ch.get('loc')}")
    